In [1]:
# import packages

import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# read train dataset

data = pd.read_csv('train.csv', index_col=0)
print(f'Train size:', data.shape)
data.head()

Train size: (200000, 201)


,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [3]:
# read test dataset

etd = pd.read_csv('test.csv', index_col=0)
print(f'Test size:', etd.shape)
etd.head()

Test size: (200000, 200)


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


There are some ID_codes in test dataset that are not included in scoring, probably these are fake ID_codes made by sampling real ID_codes. If this is the case, then there is a big chance that:

1. if at least one of the ID_code's features is unique, then the ID_code is a real sample
2. if none of the ID_code's features is unique, then the ID_code is a synthetic sample.

credit to: https://www.kaggle.com/yag320/list-of-fake-samples-and-public-private-lb-split

Let's define the features.
- orig: original 200 features
- has_one: categorical features. Consider an element 'x' in 'orig'.
        if x is unique in train dataset, then
            if x is unique in train dataset only, then
                has_one=0
            if x is unique in train dataset and real sample from test dataset, then
                has_one=4
        else
            if x appears at least another time in train dataset with target==0,
                but doesn't appear in train dataset with target==1, then
                has_one=1
            else if x appears at least another time in train dataset with target==1,
                but doesn't appear in train dataset with target==0, then
                has_one=2
            else (x appears at least two more time in both train dataset,
                with target==0 and target==1), then
                has_one=3
- has_zero: categorical features. Consider an element 'x' in 'orig'.
        if x appears at least another time in train dataset with target==0, then
            has_zero=1
        else
            has_zero=0
- not_u: numerical features. Consider an element 'x' in 'orig'.
        if x is unique in train dataset, then
            not_u=mean(train dataset)
        else
            not_u=x

In [4]:
# initiate defined features

orig = [f'var_{i}' for i in range(200)]
has_one = [f'var_{i}_has_one' for i in range(200)]
has_zero = [f'var_{i}_has_zero' for i in range(200)]
not_u = [f'var_{i}_not_unique' for i in range(200)]

In [5]:
# for each original feature 'f' in test dataset, 
# make another binary feature that says 
# whether each ID_code's element in feature 'f' is unique

# let's call these features as 'uniqueness features'

for f in tqdm(orig):
    unique_v = etd[f].value_counts()
    unique_v = unique_v.index[unique_v == 1]
    etd[f + '_u'] = etd[f].isin(unique_v)
    
print(f'Test size after adding uniqueness features:', etd.shape)
etd.head()

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:13<00:00, 15.01it/s]


Test size after adding uniqueness features: (200000, 400)


,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190_u,var_191_u,var_192_u,var_193_u,var_194_u,var_195_u,var_196_u,var_197_u,var_198_u,var_199_u
ID_code,,,,,,,,,,,,,,,,,,,,,
test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,False,False,False,False,False,False,False,False,False,False
test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,False,False,False,False,False,False,False,False,False,False
test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,False,False,False,False,False,False,False,False,False,False
test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,False,False,False,False,False,False,True,True,False,False
test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,False,False,False,False,False,False,False,False,False,False


In [6]:
# make a feature 'has_unique' in test dataset,
# stating whether each ID_code has a 'True' in 'uniqueness features'

etd['has_unique'] = etd[[f + '_u' for f in orig]].any(axis=1)
print(f'Number of real ID_codes:', etd['has_unique'].sum())

Number of real ID_codes: 100000


In [7]:
# choose ID_codes from test dataset that have unique values corresponding to a feature,
# then take only original features of those ID_codes

# let's call this new dataframe as 'real_samples'

real_samples = etd.loc[etd['has_unique'], orig]

In [8]:
# concatenate train dataset with 'real_samples' to make 'ref'

ref = pd.concat([data, real_samples], axis=0, sort=True)
print(f'All real dataset size:', ref.shape)

All real dataset size: (300000, 201)


In [9]:
# for each original feature 'f' in train dataset

for f in tqdm(orig):

    # initialize new features
    data[f + '_has_one'] = 0
    data[f + '_has_zero'] = 0
    
    # f_1_1: for each ID_code with target==1, select its element of feature 'f' with non-unique occurences in 'f'
    # f_0_1: for each ID_code with target==1, select its element of feature 'f'
    f_1 = data.loc[data['target'] == 1, f].value_counts()
    f_1_1 = set(f_1.index[f_1 > 1])
    f_0_1 = set(f_1.index[f_1 > 0])

    # f_0_0: for each ID_code with target==0, select its element of feature 'f' with non-unique occurences in 'f'
    # f_1_0: for each ID_code with target==0, select its element of feature 'f'
    f_0 = data.loc[data['target'] == 0, f].value_counts()
    f_0_0 = set(f_0.index[f_0 > 1])
    f_1_0 = set(f_0.index[f_0 > 0])
    
    # for an element in feature 'f', if the same element occurs in other ID_code that has target==1, then has_one=1
    # otherwise, has_one=0
    data.loc[data['target'] == 1, f + '_has_one'] = data.loc[data['target'] == 1, f].isin(f_1_1).astype(int)
    data.loc[data['target'] == 0, f + '_has_one'] = data.loc[data['target'] == 0, f].isin(f_0_1).astype(int)

    # for an element in feature 'f', if the same element occurs in other ID_code that has target==0, then has_zero=1
    # otherwise, has_zero=0
    data.loc[data['target'] == 1, f + '_has_zero'] = data.loc[data['target'] == 1, f].isin(f_1_0).astype(int)
    data.loc[data['target'] == 0, f + '_has_zero'] = data.loc[data['target'] == 0, f].isin(f_0_0).astype(int)

# revise the definition of has_one as explained above
data.loc[:, has_one] = 2*data.loc[:, has_one].values + data.loc[:, has_zero].values

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [05:03<00:00,  1.52s/it]


In [10]:
# analog for test dataset

for f in tqdm(orig):
    etd[f + '_has_one'] = 0
    etd[f + '_has_zero'] = 0
    f_1 = data.loc[data['target'] == 1, f].unique()
    f_0 = data.loc[data['target'] == 0, f].unique()
    etd.loc[:, f + '_has_one'] = etd[f].isin(f_1).astype(int)
    etd.loc[:, f + '_has_zero'] = etd[f].isin(f_0).astype(int)
    
etd.loc[:, has_one] = 2*etd.loc[:, has_one].values + etd.loc[:, has_zero].values

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [03:47<00:00,  1.14s/it]


In [11]:
# for each original feature 'f' in ref dataset

for f in tqdm(orig):
    
    # obtain elements with non-unique occurences in 'f'
    v = ref[f].value_counts()
    non_unique_v = v.index[v != 1]
    
    # replace the elements with unique occurences in 'f' in train dataset
    # by the average of all elements of 'f' in train dataset,
    # then assign this change to a new feature 'not_unique'
    m_trd = data[f].isin(non_unique_v)
    data[f + '_not_unique'] = m_trd  * data[f] + (~m_trd) * data[f].mean()
    
    # replace the elements with unique occurences in 'f' in test dataset
    # by the average of all elements of 'f' in train dataset
    # then assign this change to a new feature 'not_unique'
    m_etd = etd[f].isin(non_unique_v)
    etd[f + '_not_unique'] = m_etd  * etd[f] + (~m_etd) * data[f].mean()
    
    # replace elements of has_one features from has_one=0 to has_one=4
    # if those elements are also unique in 'f' of ref
    data.loc[~m_trd, f + '_has_one'] = 4
    etd.loc[~m_etd, f + '_has_one'] = 4

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [10:10<00:00,  3.05s/it]


In [12]:
# sanity check the data

print(data['var_0_has_one'].value_counts())
print(data['var_0_has_zero'].value_counts())
print(etd['var_0_has_one'].value_counts())
print(etd['var_0_has_zero'].value_counts())
print(data.shape)
print(etd.shape)

1    126381
3     27409
4     23064
0     17291
2      5855
Name: var_0_has_one, dtype: int64
1    153790
0     46210
Name: var_0_has_zero, dtype: int64
1    125826
3     28183
4     23096
0     17021
2      5874
Name: var_0_has_one, dtype: int64
1    154009
0     45991
Name: var_0_has_zero, dtype: int64
(200000, 801)
(200000, 1001)


In [40]:
data.loc[:, ['target', 'var_0', 'var_0_has_zero', 'var_0_has_one', 'var_0_not_unique']].head()

,target,var_0,var_0_has_zero,var_0_has_one,var_0_not_unique
ID_code,,,,,
train_0,0,8.9255,1,1,8.9255
train_1,0,11.5006,1,1,11.5006
train_2,0,8.6093,1,1,8.6093
train_3,0,11.0604,1,1,11.0604
train_4,0,9.8369,1,1,9.8369


In [41]:
etd.loc[:, ['var_0', 'var_0_u', 'has_unique', 'var_0_has_zero', 'var_0_has_one', 'var_0_not_unique']].head()

,var_0,var_0_u,has_unique,var_0_has_zero,var_0_has_one,var_0_not_unique
ID_code,,,,,,
test_0,11.0656,False,False,1,1,11.0656
test_1,8.5304,False,False,1,3,8.5304
test_2,5.4827,False,False,1,1,5.4827
test_3,8.5374,True,True,1,1,8.5374
test_4,11.7058,False,False,1,1,11.7058


In [43]:
# save datasets to local

pd.DataFrame.to_csv(data.reset_index(), '921_data.csv')
pd.DataFrame.to_csv(etd.reset_index(), '921_etd.csv')

In [1]:
# import packages

import fastai
from fastai.tabular import *
from fastai.text import *
from fastai.callbacks import SaveModelCallback

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from tqdm import tqdm
import logging
import gc

In [2]:
#logger

def get_logger():
    FORMAT = '[%(levelname)s]%(asctime)s:%(name)s:%(message)s'
    logging.basicConfig(format=FORMAT)
    logger = logging.getLogger('main')
    logger.setLevel(logging.DEBUG)
    return logger
    
logger = get_logger()

def auroc_score(input, target):
    input, target = input.cpu().numpy()[:,1], target.cpu().numpy()
    return roc_auc_score(target, input)

In [3]:
# callback to calculate AUC at the end of each epoch

class AUROC(Callback):
    _order = -20 # needs to run before the recorder

    def __init__(self, learn, **kwargs):
        self.learn = learn
        self.output = []
        self.target = []
        
    def on_train_begin(self, **kwargs): self.learn.recorder.add_metric_names(['AUROC'])
        
    def on_epoch_begin(self, **kwargs): self.output, self.target = [], []
    
    def on_batch_end(self, last_target, last_output, train, **kwargs):
        if not train:
            self.output.append(last_output)
            self.target.append(last_target)
                
    def on_epoch_end(self, last_metrics, **kwargs):
        if len(self.output) > 0:
            output = torch.cat(self.output)
            target = torch.cat(self.target)
            preds = F.softmax(output, dim=1)
            metric = auroc_score(preds, target)
            return add_metrics(last_metrics, [metric])

In [4]:
# callback that do the shuffle augmentation        

class AugShuffCallback(LearnerCallback):
    def __init__(self, learn:Learner):
        super().__init__(learn)
        
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        if not train: return
        m_pos = last_target==1
        m_neg = last_target==0
        
        pos_cat = last_input[0][m_pos]
        pos_cont = last_input[1][m_pos]
        
        neg_cat = last_input[0][m_neg]
        neg_cont = last_input[1][m_neg]
        
        for f in range(200):
            shuffle_pos = torch.randperm(pos_cat.size(0)).to(last_input[0].device)
            pos_cat[:,f] = pos_cat[shuffle_pos,f]
            pos_cont[:,f] = pos_cont[shuffle_pos, f]
            pos_cont[:,f+200] = pos_cont[shuffle_pos, f+200]
            
            shuffle_neg = torch.randperm(neg_cat.size(0)).to(last_input[0].device)
            neg_cat[:,f] = neg_cat[shuffle_neg,f]
            neg_cont[:, f] = neg_cont[shuffle_neg, f]
            neg_cont[:,f+200] = neg_cont[shuffle_neg, f+200]
        
        new_input = [torch.cat([pos_cat, neg_cat]), torch.cat([pos_cont, neg_cont])]
        new_target = torch.cat([last_target[m_pos], last_target[m_neg]])
        
        return {'last_input': new_input, 'last_target': new_target}

In [5]:
# a longer version of the random sampler : each samples is given "mult" times.

class LongerRandomSampler(Sampler):
    def __init__(self, data_source, replacement=False, num_samples=None, mult=3):
        self.data_source = data_source
        self.replacement = replacement
        self.num_samples = num_samples
        self.mult = mult

        if self.num_samples is not None and replacement is False:
            raise ValueError("With replacement=False, num_samples should not be specified, "
                             "since a random permute will be performed.")

        if self.num_samples is None:
            self.num_samples = len(self.data_source) * self.mult

        if not isinstance(self.num_samples, int) or self.num_samples <= 0:
            raise ValueError("num_samples should be a positive integeral "
                             "value, but got num_samples={}".format(self.num_samples))
        if not isinstance(self.replacement, bool):
            raise ValueError("replacement should be a boolean value, but got "
                             "replacement={}".format(self.replacement))

    def __iter__(self):
        n = len(self.data_source)
        if self.replacement:
            return iter(torch.randint(high=n, size=(self.num_samples*self.mult,), dtype=torch.int64).tolist())
        return iter(torch.randperm(n).tolist()*self.mult)

    def __len__(self):
        return len(self.data_source)*self.mult

In [6]:
# the NN structure, starting from fast.ai TabularModel

class my_TabularModel(nn.Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs:ListSizes, n_cont:int, out_sz:int, layers:Collection[int], ps:Collection[float]=None,
                 emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, bn_final:bool=False, 
                 cont_emb=2, cont_emb_notu=2):
        
        super().__init__()
        
        # continuous embedding NN for raw features
        self.cont_emb = cont_emb[1]
        self.cont_emb_l = torch.nn.Linear(1 + 2, cont_emb[0])
        self.cont_emb_l2 = torch.nn.Linear(cont_emb[0], cont_emb[1])
        
        # continuous embedding NN for "not unique" features", cf #1 solution post
        self.cont_emb_notu_l = torch.nn.Linear(1 + 2, cont_emb_notu[0])
        self.cont_emb_notu_l2 = torch.nn.Linear(cont_emb_notu[0], cont_emb_notu[1])
        self.cont_emb_notu = cont_emb_notu[1]
            
        ps = ifnone(ps, [0]*len(layers))
        ps = listify(ps, layers)
        
        # embedding for "has one" categorical features, cf #1 solution post
        self.embeds = embedding(emb_szs[0][0], emb_szs[0][1])
        
        # at first we included information about the variable being processed (to extract feature importance)
        # it works better using a constant feat (kind of intercept)
        self.embeds_feat = embedding(201, 2)
        self.embeds_feat_w = embedding(201, 2)
        
        self.emb_drop = nn.Dropout(emb_drop)
        
        n_emb = self.embeds.embedding_dim
        n_emb_feat = self.embeds_feat.embedding_dim
        n_emb_feat_w = self.embeds_feat_w.embedding_dim
        
        self.n_emb, self.n_emb_feat, self.n_emb_feat_w, self.n_cont,self.y_range = n_emb, n_emb_feat, n_emb_feat_w, n_cont, y_range
        
        sizes = self.get_sizes(layers, out_sz)
        actns = [nn.ReLU(inplace=True)] * (len(sizes)-2) + [None]
        layers = []
        for i,(n_in,n_out,dp,act) in enumerate(zip(sizes[:-1],sizes[1:],[0.]+ps,actns)):
            layers += bn_drop_lin(n_in, n_out, bn=use_bn and i!=0, p=dp, actn=act)
            
        self.layers = nn.Sequential(*layers)
        self.seq = nn.Sequential()
        
        # input size for the NN that predicts weights
        inp_w = self.n_emb + self.n_emb_feat_w + self.cont_emb + self.cont_emb_notu
        # input size for the final NN that predicts output
        inp_x = self.n_emb + self.cont_emb + self.cont_emb_notu
        
        # NN that predicts the weights
        self.weight = nn.Linear(inp_w, 5)
        self.weight2 = nn.Linear(5,1)
        
        mom = 0.1
        self.bn_cat = nn.BatchNorm1d(200, momentum=mom)
        self.bn_feat_emb = nn.BatchNorm1d(200, momentum=mom)
        self.bn_feat_w = nn.BatchNorm1d(200, momentum=mom)
        self.bn_raw = nn.BatchNorm1d(200, momentum=mom)
        self.bn_notu = nn.BatchNorm1d(200, momentum=mom)
        self.bn_w = nn.BatchNorm1d(inp_w, momentum=mom)
        self.bn = nn.BatchNorm1d(inp_x, momentum=mom)
        
    def get_sizes(self, layers, out_sz):
        return [self.n_emb + self.cont_emb_notu + self.cont_emb] + layers + [out_sz]

    def forward(self, x_cat:Tensor, x_cont:Tensor) -> Tensor:
        b_size = x_cont.size(0)
        
        # embedding of has one feat
        x = [self.embeds(x_cat[:,i]) for i in range(200)]
        x = torch.stack(x, dim=1)
        
        # embedding of intercept. It was embedding of feature id before
        x_feat_emb = self.embeds_feat(x_cat[:,200])
        x_feat_emb = torch.stack([x_feat_emb]*200, 1)
        x_feat_emb = self.bn_feat_emb(x_feat_emb)
        x_feat_w = self.embeds_feat_w(x_cat[:,200])
        x_feat_w = torch.stack([x_feat_w]*200, 1)
        
        # "continuous embedding" of raw features
        x_cont_raw = x_cont[:,:200].contiguous().view(-1, 1)
        x_cont_raw = torch.cat([x_cont_raw, x_feat_emb.view(-1, self.n_emb_feat)], 1)
        x_cont_raw = F.relu(self.cont_emb_l(x_cont_raw))
        x_cont_raw = self.cont_emb_l2(x_cont_raw)
        x_cont_raw = x_cont_raw.view(b_size, 200, self.cont_emb)
        
        # "continuous embedding" of not unique features
        x_cont_notu = x_cont[:,200:].contiguous().view(-1, 1)
        x_cont_notu = torch.cat([x_cont_notu, x_feat_emb.view(-1,self.n_emb_feat)], 1)
        x_cont_notu = F.relu(self.cont_emb_notu_l(x_cont_notu))
        x_cont_notu = self.cont_emb_notu_l2(x_cont_notu)
        x_cont_notu = x_cont_notu.view(b_size, 200, self.cont_emb_notu)

        x_cont_notu = self.bn_notu(x_cont_notu)
        x = self.bn_cat(x)
        x_cont_raw = self.bn_raw(x_cont_raw)

        x = self.emb_drop(x)
        x_cont_raw = self.emb_drop(x_cont_raw)
        x_cont_notu = self.emb_drop(x_cont_notu)
        x_feat_w = self.bn_feat_w(x_feat_w)
        
        # predict a weight for each of the previous embeddings
        x_w = torch.cat([x.view(-1,self.n_emb),
                         x_feat_w.view(-1,self.n_emb_feat_w),
                         x_cont_raw.view(-1, self.cont_emb), 
                         x_cont_notu.view(-1, self.cont_emb_notu)], 1)

        x_w = self.bn_w(x_w)

        w = F.relu(self.weight(x_w))
        w = self.weight2(w).view(b_size, -1)
        w = torch.nn.functional.softmax(w, dim=-1).unsqueeze(-1)

        # weighted average of the differents embeddings using weights given by NN
        x = (w * x).sum(dim=1)
        x_cont_raw = (w * x_cont_raw).sum(dim=1)
        x_cont_notu = (w * x_cont_notu).sum(dim=1)
        
        # Use NN on the weighted average to predict final output
        x = torch.cat([x, x_cont_raw, x_cont_notu], 1) if self.n_emb != 0 else x_cont
        x = self.bn(x)
            
        x = self.seq(x)
        x = self.layers(x)
        return x

In [7]:
# seed

def set_seed(seed=42):
    # python RNG
    random.seed(seed)

    # pytorch RNGs
    import torch
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

    # numpy RNG
    import numpy as np
    np.random.seed(seed)

In [8]:
# load datasets and set up features

logger.info('Input data')

data = pd.read_csv('921_data.csv')
data = data.set_index('ID_code')

etd = pd.read_csv('921_etd.csv')
etd = etd.set_index('ID_code')

has_one = [f'var_{i}_has_one' for i in range(200)]
orig = [f'var_{i}' for i in range(200)]
not_u = [f'var_{i}_not_unique' for i in range(200)]
cont_vars = orig + not_u
cat_vars = has_one
target = 'target'
path = './'

[INFO]2019-11-09 09:18:31,810:main:Input data


In [9]:
# categorical features treatment

logger.info('cat treatment')

for f in cat_vars:
    data[f] = data[f].astype('category').cat.as_ordered()
    etd[f] = pd.Categorical(etd[f], categories=data[f].cat.categories, ordered=True)

[INFO]2019-11-09 09:20:21,851:main:cat treatment


In [10]:
# constant feature to replace feature index information

feat = ['intercept']
data['intercept'] = 1
data['intercept'] = data['intercept'].astype('category')
etd['intercept'] = 1
etd['intercept'] = etd['intercept'].astype('category')
    
cat_vars += feat

In [11]:
# feature scaling

ss = StandardScaler()
data[cont_vars] = ss.fit_transform(data[cont_vars].values)
etd[cont_vars] = ss.transform(etd[cont_vars].values)

In [12]:
# feature selection

data = data[cont_vars + cat_vars + ['target']]
etd = etd[cont_vars + cat_vars]
data[target] = data[target].astype('int')
print(f'Train dataset size:', data.shape)
print(f'Test dataset size:', etd.shape)
gc.collect()

Train dataset size: (200000, 602)
Test dataset size: (200000, 601)


7

In [13]:
# split data for validation

fold_seed = 42
ss = StratifiedKFold(n_splits=5, random_state=fold_seed, shuffle=True)

folds = []
for num, (train,test) in enumerate(ss.split(data[target], data[target])):
    folds.append([train, test])

In [14]:
# set hyperparameters

layers=[32]
ps=0.2
emb_drop=0.08
cont_emb=(50,10)
cont_emb_notu=(50,10)
emb_szs = [[6,12]]
use_bn = True
joined=False
seeds = [42]

results = []
sub_preds = pd.DataFrame(columns=range(10), index=etd.index)

In [15]:
# train & predict

for num_fold, (train, test) in enumerate(folds):
    procs=[]
    df = (TabularList.from_df(data, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
              .split_by_idx(test)
              .label_from_df(cols=target)
              .add_test(TabularList.from_df(etd, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs))
              .databunch(num_workers=0, bs=1024))
            
    df.dls[0].dl = df.dls[0].new(sampler=LongerRandomSampler(data_source=df.train_ds, mult=2), shuffle=False).dl
    for num_seed, seed in enumerate(seeds):
        logger.info(f'Model {num_fold} seed {num_seed}')
        set_seed(seed)
        model = my_TabularModel(emb_szs, len(df.cont_names), out_sz=df.c, layers=layers, ps=ps, emb_drop=emb_drop,
                                y_range=None, use_bn=use_bn, cont_emb=cont_emb, cont_emb_notu=cont_emb_notu)

        learn = Learner(df, model, metrics=None, callback_fns=AUROC, wd=0.1)
        learn.fit_one_cycle(15, 
                            max_lr=1e-2, 
                            callbacks=[SaveModelCallback(learn, 
                                                         every='improvement', 
                                                         monitor='AUROC', 
                                                         name=f'fold{fold_seed}_{num_fold}_seed_{seed}'), 
                                       AugShuffCallback(learn)])
        pred, _ = learn.get_preds()
        pred = pred[:,1]
        
        pred_test, _ = learn.get_preds(DatasetType.Test)
        pred_test = pred_test[:,1]
        
        sub_preds.loc[:, num_fold] = pred_test
        # results.append(np.max(learn.recorder.metrics))
        # logger.info('result ' + str(results[-1]))
        
        np.save(f'oof_fold{fold_seed}_{num_fold}_seed_{seed}.npy', pred)
        np.save(f'test_fold{fold_seed}_{num_fold}_seed_{seed}.npy', pred_test)
        
        del learn, pred, model, pred_test; gc.collect()
    del df; gc.collect()
# print(results)
# print(np.mean(results))

sub_preds[target] = sub_preds.rank().mean(axis=1)
sub_preds[[target]].to_csv('submission_NN_wo_pseudo_seed42.csv', index_label='ID_code')